<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignment1/question1/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

q1

In [19]:
! pip install -q pyspark

In [20]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import col, sum , mean , mode , when


base_path = f'./'

#base_path = f"/opt/spark/data/{student_id}/"


In [21]:

spark = SparkSession.builder \
    .appName("assignment1_CH24M571") \
    .master("local[*]") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.core", "4") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.instances", "2")\
    .config("spark.driver.core", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")


# # spark = SparkSession.builder \: This starts the process of building a SparkSession.
# .appName("Assignment1") \: This sets the name of the Spark application to "Assignment1".
# .master("local[*]") \: This configures Spark to run locally on your machine using all available cores.
# .config("spark.executor.memory", "512m") \: This sets the amount of memory allocated to each executor (the processes that perform the actual computations) to 512 megabytes.
# .config("spark.executor.core", "4") \: This sets the number of cores used by each executor to 4.
# .config("spark.driver.memory", "2g") \: This sets the amount of memory allocated to the Spark driver (the process that coordinates the execution) to 2 gigabytes.
# .config("spark.executor.instances", "2")\: This sets the number of executor instances to 2.
# .config("spark.driver.core", "2") \: This sets the number of cores used by the driver to 2.
# .getOrCreate(): This retrieves an existing SparkSession if one is already running, or creates a new one if not.
# spark.sparkContext.setLogLevel("INFO"): This sets the logging level for the Spark context to "INFO", which means informational messages will be displayed.

In [22]:
# a timer decorator

import time
from functools import wraps

def timer_decorator(func):
  """
  A decorator that measures the execution time of a function.
  """
  @wraps(func)
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
    return result
  return wrapper

In [23]:
# setting the schemas for the 3 csv s

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, StructType

# Customers schema
customers_schema = StructType([
    StructField("customer_id", StringType(), nullable=False),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("preferences", StringType(), True),  # Will parse to ArrayType later
    StructField("address", StringType(), True)       # Will parse to StructType later
])

# Orders schema
orders_schema = StructType([
    StructField("order_id", StringType(), nullable=False),
    StructField("customer_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("items", StringType(), True),
    StructField("shipping_address", StringType(), True),
    StructField("total_amount", FloatType(), True)
])

# Products schema
products_schema = StructType([
    StructField("product_id", StringType(), nullable=False),
    StructField("name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("tags", StringType(), True)
])

In [24]:
# defining file paths

customers_csv= base_path +'customers.csv'
orders_csv = base_path +'orders.csv'
products_csv = base_path +'products.csv'

In [25]:

# read csv file
customers_df = spark.read.csv(customers_csv, header=True, schema=customers_schema,escape="\"",quote="\"")
# show the dataframe
customers_df.show()

+-----------+-----------+---+--------------------+--------------------+--------------------+
|customer_id|       name|age|               email|         preferences|             address|
+-----------+-----------+---+--------------------+--------------------+--------------------+
| CUST000001| Customer_1| 61|customer1@example...|["sports", "trave...|{"street": "2796 ...|
| CUST000002| Customer_2| 55|customer2@example...|                NULL|{"street": "5409 ...|
| CUST000003| Customer_3| 79|customer3@example...|["movies", "books...|{"street": "6081 ...|
| CUST000004| Customer_4| 73|customer4@example...|["books", "fitnes...|{"street": "3303 ...|
| CUST000005| Customer_5| 80|customer5@example...|["music", "books"...|{"street": "4864 ...|
| CUST000006| Customer_6| 44|customer6@example...|["travel", "fitne...|{"street": "3592 ...|
| CUST000007| Customer_7| 57|customer7@example...|["music", "movies...|{"street": "2932 ...|
| CUST000008| Customer_8| 20|                NULL|["books", "fitnes...

In [26]:
# read csv file
orders_df = spark.read.csv(orders_csv,header=True, schema=orders_schema,escape="\"",quote="\"")
# show the dataframe
orders_df.show()

+----------+-----------+----------+--------------------+--------------------+------------+
|  order_id|customer_id|order_date|               items|    shipping_address|total_amount|
+----------+-----------+----------+--------------------+--------------------+------------+
|ORD0000001| CUST000046|2023-08-16|[{"product_id": "...|{"street": "703 M...|     1112.73|
|ORD0000002| CUST000027|2023-11-20|[{"product_id": "...|{"street": "6503 ...|      456.04|
|ORD0000003| CUST000093|2023-11-27|[{"product_id": "...|{"street": "9664 ...|     1420.89|
|ORD0000004| CUST000017|2023-04-08|[{"product_id": "...|{"street": "1329 ...|     1450.66|
|ORD0000005| CUST000092|2023-09-24|[{"product_id": "...|{"street": "3075 ...|     2016.47|
|ORD0000006| CUST000029|2023-09-04|[{"product_id": "...|{"street": "176 M...|      800.88|
|ORD0000007| CUST000093|2023-10-22|[{"product_id": "...|{"street": "9664 ...|        NULL|
|ORD0000008| CUST000094|2023-10-20|[{"product_id": "...|{"street": "4391 ...|      121.35|

In [27]:
# read csv file
products_df = spark.read.csv(products_csv, header=True, schema=products_schema,escape="\"",quote="\"")
# show the dataframe
products_df.show()

+----------+----------+-----------+------+--------------------+
|product_id|      name|   category| price|                tags|
+----------+----------+-----------+------+--------------------+
| PROD00001| Product_1|electronics|  5.32|["new", "sale", "...|
| PROD00002| Product_2|     sports| 59.11|["limited", "eco-...|
| PROD00003| Product_3|     sports|370.91|             ["new"]|
| PROD00004| Product_4|     sports| 69.84|            ["sale"]|
| PROD00005| Product_5|     sports| 135.9|["sale", "new", "...|
| PROD00006| Product_6|electronics|428.33|["new", "limited"...|
| PROD00007| Product_7|electronics|400.44|         ["popular"]|
| PROD00008| Product_8|     sports| 40.45|["limited", "popu...|
| PROD00009| Product_9|     sports|187.57|["limited", "new"...|
| PROD00010|Product_10|  furniture| 176.2|  ["popular", "new"]|
+----------+----------+-----------+------+--------------------+



- Print the size of each dataframe and schema

In [28]:
print(f"Rows: {customers_df.count()}")
print(f"Columns: {len(customers_df.columns)}")
customers_df.printSchema()
print("-------------------------------------")
print(f"Rows: {orders_df.count()}")
print(f"Columns: {len(orders_df.columns)}")
orders_df.printSchema()
print("-------------------------------------")
print(f"Rows: {products_df.count()}")
print(f"Columns: {len(products_df.columns)}")
products_df.printSchema()
print("-------------------------------------")

Rows: 100
Columns: 6
root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- preferences: string (nullable = true)
 |-- address: string (nullable = true)

-------------------------------------
Rows: 500
Columns: 6
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- items: string (nullable = true)
 |-- shipping_address: string (nullable = true)
 |-- total_amount: float (nullable = true)

-------------------------------------
Rows: 10
Columns: 5
root
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: float (nullable = true)
 |-- tags: string (nullable = true)

-------------------------------------


-
Identify the corrupt values in each file and report the number of corrupt rows and
total corrupt values in each file.

In [29]:
# Check for null values in customers_df

# CHECKING JUST THE IDS - IDEALLY THERE SHOULDNT BE NONE

customers_null_counts = customers_df.select(
    sum(col("customer_id")
    .isNull()         # check if each value in column is null , if so cast the value as 0 or 1
    .cast("integer"))
    .alias("customer_id_nulls")).collect()[0]  # aggregate and put into a new column

    #
print(f"Customers DataFrame - Nulls in customer_id: {customers_null_counts['customer_id_nulls']}")

# Check for null values in orders_df

orders_null_counts = orders_df.select(
    sum(col("order_id").isNull().cast("integer")).alias("order_id_nulls"),
    sum(col("customer_id").isNull().cast("integer")).alias("customer_id_nulls")
).collect()[0]

print(f"Orders DataFrame - Nulls in order_id: {orders_null_counts['order_id_nulls']}")
print(f"Orders DataFrame - Nulls in customer_id: {orders_null_counts['customer_id_nulls']}")


# Check for null values in products_df
products_null_counts = products_df.select(sum(col("product_id").isNull().cast("integer")).alias("product_id_nulls")).collect()[0]
print(f"Products DataFrame - Nulls in product_id: {products_null_counts['product_id_nulls']}")

Customers DataFrame - Nulls in customer_id: 0
Orders DataFrame - Nulls in order_id: 0
Orders DataFrame - Nulls in customer_id: 0
Products DataFrame - Nulls in product_id: 0


In [30]:

# Check for null values in all columns of customers_df

print("Null counts for customers_df:")

customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

# loop over all columns and check for null values

# Check for null values in all columns of orders_df
print("Null counts for orders_df:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

# Check for null values in all columns of products_df
print("Null counts for products_df:")
products_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in products_df.columns)).show()

Null counts for customers_df:
+-----------+----+---+-----+-----------+-------+
|customer_id|name|age|email|preferences|address|
+-----------+----+---+-----+-----------+-------+
|          0|   0|  1|    5|          5|      0|
+-----------+----+---+-----+-----------+-------+

Null counts for orders_df:
+--------+-----------+----------+-----+----------------+------------+
|order_id|customer_id|order_date|items|shipping_address|total_amount|
+--------+-----------+----------+-----+----------------+------------+
|       0|          0|         0|    0|              23|          14|
+--------+-----------+----------+-----+----------------+------------+

Null counts for products_df:
+----------+----+--------+-----+----+
|product_id|name|category|price|tags|
+----------+----+--------+-----+----+
|         0|   0|       0|    0|   0|
+----------+----+--------+-----+----+



For numerical columns, fill the missing values with mean value

In [31]:
from pyspark.sql.functions import mean

# age and total_amount are the only numerical columns
# price has no nulls


# mean 'age' column in customers_df
mean_age = customers_df.select(mean(col("age"))).collect()[0][0]

# Fill null values in the 'age' column with the calculated mean
customers_df = customers_df.na.fill(mean_age, subset=["age"])

print("-----------------")

# Calculate the mean of the 'total_amount' column in orders_df
mean_total_amount = orders_df.select(mean(col("total_amount"))).collect()[0][0]

# Fill null values in the 'total_amount' column with the calculated mean
orders_df = orders_df.na.fill(mean_total_amount, subset=["total_amount"])

print("-----------------")

mean_price = products_df.select(mean(col("price"))).collect()[0][0]

# Fill null values in the 'price' column with the calculated mean
products_df = products_df.na.fill(mean_price, subset=["price"])


print("Null counts for customers_df after filling 'age' nulls:")
customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

print("Null counts for orders_df after filling 'total_amount' nulls:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

# doing for price just in case
print("Null counts for products_df after filling 'price' nulls:")
products_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in products_df.columns)).show()

-----------------
-----------------
Null counts for customers_df after filling 'age' nulls:
+-----------+----+---+-----+-----------+-------+
|customer_id|name|age|email|preferences|address|
+-----------+----+---+-----+-----------+-------+
|          0|   0|  0|    5|          5|      0|
+-----------+----+---+-----+-----------+-------+

Null counts for orders_df after filling 'total_amount' nulls:
+--------+-----------+----------+-----+----------------+------------+
|order_id|customer_id|order_date|items|shipping_address|total_amount|
+--------+-----------+----------+-----+----------------+------------+
|       0|          0|         0|    0|              23|           0|
+--------+-----------+----------+-----+----------------+------------+

Null counts for products_df after filling 'price' nulls:
+----------+----+--------+-----+----+
|product_id|name|category|price|tags|
+----------+----+--------+-----+----+
|         0|   0|       0|    0|   0|
+----------+----+--------+-----+----+



- For categorical or string columns, fill the missing values with most frequent value.

In [32]:
def categorical_cols(customers_df, orders_df, products_df):
  ''' will return a tuple of list for each input df '''

  string_cols_customers = [c for c, t in customers_df.dtypes if t == 'string']
  string_cols_orders = [c for c, t in orders_df.dtypes if t == 'string']
  string_cols_products = [c for c, t in products_df.dtypes if t == 'string']

  return string_cols_customers, string_cols_orders, string_cols_products


In [33]:
string_cols_customers, string_cols_orders, string_cols_products = categorical_cols(customers_df, orders_df, products_df)

# for col_name in string_cols_customers:
#     # Calculate and fill nulls with the mode value
#     mode_result = customers_df.groupBy(col_name).count().orderBy("count", ascending=False).first()
#     if mode_result is not None:
#         mode_value = mode_result[0]
#         customers_df = customers_df.na.fill(mode_value, subset=[col_name])
#     else:
#         print(f"Skipping fill for column '{col_name}' in customers_df as mode is None (column might be all nulls).")


# customers_df.show(3)

In [34]:

def fill_nulls_with_mean_or_mode(df):
    """
    Fills null values in a DataFrame with the mean for numerical columns
    and the mode for string columns.
    """
    # Calculate null counts for each column
    null_counts = df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in df.columns)).collect()[0]

    for col_name in df.columns:
        null_count = null_counts[col_name]
        # Check if the column has null values
        if(null_count != 0):
            print(f"In DataFrame column '{col_name}' has {null_count} null values. Filling the null values with {'mean' if df.schema[col_name].dataType in [IntegerType(), FloatType()] else 'mode'}.")

            # Filling the null values in the string type column with the mode
            if df.schema[col_name].dataType == StringType():
                mode_col = df.agg(mode(col(col_name))).collect()[0][0]
                # Check if mode_col is not None before filling
                if mode_col is not None:
                  df = df.withColumn(col_name, when(col(col_name).isNull(), mode_col).otherwise(col(col_name)))
                else:
                  print(f"Skipping fill for column '{col_name}' as mode is None.")


            # Filling the null values in the integer type column with the mean
            elif df.schema[col_name].dataType == IntegerType():
                mean_col = df.agg(mean(col(col_name))).collect()[0][0]
                # Check if mean_col is not None before filling
                if mean_col is not None:
                  df = df.withColumn(col_name, when(col(col_name).isNull(), mean_col).otherwise(col(col_name)))
                else:
                  print(f"Skipping fill for column '{col_name}' as mean is None.")

            # Filing the null values in the float type column with the mean
            elif df.schema[col_name].dataType == FloatType():
                mean_col = df.agg(mean(col(col_name))).collect()[0][0]
                # Check if mean_col is not None before filling
                if mean_col is not None:
                  df = df.withColumn(col_name, when(col(col_name).isNull(), mean_col).otherwise(col(col_name)))
                else:
                  print(f"Skipping fill for column '{col_name}' as mean is None.")

    return df

# Apply the function to each dataframe
customers_df = fill_nulls_with_mean_or_mode(customers_df)
orders_df = fill_nulls_with_mean_or_mode(orders_df)
products_df = fill_nulls_with_mean_or_mode(products_df)


print("Null counts for customers_df after filling nulls:")
customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

print("Null counts for orders_df after filling nulls:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

print("Null counts for products_df after filling nulls:")
products_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in products_df.columns)).show()

In DataFrame column 'email' has 5 null values. Filling the null values with mode.
In DataFrame column 'preferences' has 5 null values. Filling the null values with mode.
In DataFrame column 'shipping_address' has 23 null values. Filling the null values with mode.
Null counts for customers_df after filling nulls:
+-----------+----+---+-----+-----------+-------+
|customer_id|name|age|email|preferences|address|
+-----------+----+---+-----+-----------+-------+
|          0|   0|  0|    0|          0|      0|
+-----------+----+---+-----+-----------+-------+

Null counts for orders_df after filling nulls:
+--------+-----------+----------+-----+----------------+------------+
|order_id|customer_id|order_date|items|shipping_address|total_amount|
+--------+-----------+----------+-----+----------------+------------+
|       0|          0|         0|    0|               0|           0|
+--------+-----------+----------+-----+----------------+------------+

Null counts for products_df after filling 

Compute total spending by each customer

In [36]:
# Total spending by each customer
customer_spending = orders_df.groupBy("customer_id").agg(sum("total_amount").alias("total_spending"))
customer_spending = customer_spending.orderBy("customer_id")
print("Total spending by each customer:")
customer_spending.show()

Total spending by each customer:
+-----------+------------------+
|customer_id|    total_spending|
+-----------+------------------+
| CUST000001| 7837.950028419495|
| CUST000002| 6334.889953613281|
| CUST000003| 8870.059913635254|
| CUST000004| 4024.570053100586|
| CUST000005| 6235.429955482483|
| CUST000006|10116.099697589874|
| CUST000007|4060.5299305915833|
| CUST000008|            3159.5|
| CUST000009|  4424.84614944458|
| CUST000010| 2295.570068359375|
| CUST000011| 5725.100067138672|
| CUST000012| 2700.850028991699|
| CUST000013| 8379.286041259766|
| CUST000014|2717.8399658203125|
| CUST000015|4083.1600646972656|
| CUST000016|13439.240074157715|
| CUST000017| 7496.780090332031|
| CUST000018|1528.8999633789062|
| CUST000019| 7249.889976501465|
| CUST000020|  1283.52001953125|
+-----------+------------------+
only showing top 20 rows



Identify top 3 products per customer by quantity

In [44]:
from pyspark.sql.functions import explode, split, regexp_extract, rank, from_json, col, sum as sum_, struct
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window

# Define the schema for the items within the JSON string
item_schema = ArrayType(
    StructType([
        StructField("product_id", StringType(), True),
        StructField("quantity", IntegerType(), True),
        StructField("price", FloatType(), True) # Include price in schema as it's in the JSON
    ])
)

# Parse the 'items' column from JSON string to a structured type
orders_with_parsed_items = orders_df.withColumn("parsed_items", from_json(col("items"), item_schema))

# Explode the 'parsed_items' array to get one row per item
order_items_df = orders_with_parsed_items.withColumn("item", explode(col("parsed_items"))) \
    .select("customer_id", col("item.product_id").alias("product_id"), col("item.quantity").alias("quantity"))

# Group by customer and product to get total quantity per customer-product pair
customer_product_quantity = order_items_df.groupBy("customer_id", "product_id") \
    .agg(sum_("quantity").alias("total_quantity"))

# Define a window function to rank products within each customer
window_spec = Window.partitionBy("customer_id").orderBy(col("total_quantity").desc())

# Apply the window function to rank products
ranked_products = customer_product_quantity.withColumn("rank", rank().over(window_spec))

# Filter for the top 3 products per customer
top_3_products_per_customer = ranked_products.filter(col("rank") <= 3)

print("Top 3 products per customer by quantity:")
top_3_products_per_customer.show()

Top 3 products per customer by quantity:
+-----------+----------+--------------+----+
|customer_id|product_id|total_quantity|rank|
+-----------+----------+--------------+----+
| CUST000001| PROD00002|            12|   1|
| CUST000001| PROD00001|            11|   2|
| CUST000001| PROD00003|             7|   3|
| CUST000002| PROD00009|             8|   1|
| CUST000002| PROD00007|             6|   2|
| CUST000002| PROD00001|             5|   3|
| CUST000002| PROD00010|             5|   3|
| CUST000003| PROD00004|            12|   1|
| CUST000003| PROD00005|             7|   2|
| CUST000003| PROD00001|             7|   2|
| CUST000004| PROD00002|             6|   1|
| CUST000004| PROD00010|             5|   2|
| CUST000004| PROD00007|             3|   3|
| CUST000004| PROD00006|             3|   3|
| CUST000004| PROD00001|             3|   3|
| CUST000005| PROD00008|            10|   1|
| CUST000005| PROD00003|             6|   2|
| CUST000005| PROD00005|             6|   2|
| CUST000005| 

Create a summary dataframe with
•customer id
•total orders
•total spent
•top 3 product ids

In [45]:
from pyspark.sql.functions import collect_list, concat_ws

# Calculate total orders per customer
customer_total_orders = orders_df.groupBy("customer_id").count().withColumnRenamed("count", "total_orders")

# Join total orders and total spending
summary_df = customer_total_orders.join(customer_spending, "customer_id")

# Collect the top 3 product IDs into a list for each customer
top_products_list = top_3_products_per_customer.groupBy("customer_id") \
    .agg(collect_list("product_id").alias("top_product_ids"))

# Join the summary dataframe with the top products list
summary_df = summary_df.join(top_products_list, "customer_id")

# Display the summary dataframe
print("Summary DataFrame:")
summary_df.show(truncate=False)

Summary DataFrame:
+-----------+------------+------------------+-------------------------------------------------------+
|customer_id|total_orders|total_spending    |top_product_ids                                        |
+-----------+------------+------------------+-------------------------------------------------------+
|CUST000084 |4           |5978.809997558594 |[PROD00004, PROD00005, PROD00003]                      |
|CUST000082 |3           |4824.7000732421875|[PROD00003, PROD00001, PROD00002, PROD00007, PROD00004]|
|CUST000007 |5           |4060.5299305915833|[PROD00004, PROD00006, PROD00008]                      |
|CUST000015 |4           |4083.1600646972656|[PROD00003, PROD00009, PROD00008, PROD00002]           |
|CUST000099 |7           |8306.859985351562 |[PROD00003, PROD00005, PROD00007]                      |
|CUST000075 |5           |6893.740081787109 |[PROD00005, PROD00001, PROD00003, PROD00007]           |
|CUST000085 |6           |4631.476043701172 |[PROD00001, PROD00

Display top 50 customers in terms of total orders

In [46]:
# Order the summary DataFrame by total orders in descending order
top_50_customers_by_orders = summary_df.orderBy(col("total_orders").desc())

# Display the top 50 customers
print("Top 50 customers in terms of total orders:")
top_50_customers_by_orders.show(50, truncate=False)

Top 50 customers in terms of total orders:
+-----------+------------+------------------+------------------------------------------------------------------+
|customer_id|total_orders|total_spending    |top_product_ids                                                   |
+-----------+------------+------------------+------------------------------------------------------------------+
|CUST000033 |12          |14896.74008178711 |[PROD00002, PROD00001, PROD00005]                                 |
|CUST000083 |11          |12215.219970703125|[PROD00009, PROD00004, PROD00006, PROD00003, PROD00002]           |
|CUST000006 |10          |10116.099697589874|[PROD00009, PROD00006, PROD00010]                                 |
|CUST000003 |9           |8870.059913635254 |[PROD00004, PROD00005, PROD00001]                                 |
|CUST000016 |9           |13439.240074157715|[PROD00003, PROD00006, PROD00007]                                 |
|CUST000001 |9           |7837.950028419495 |[PROD000